# Creating Local RAG Pipeline from Scratch

## 1. Data Preparation and Embedding Creation

### 1.1 Importing PDF Document for our Book

In [4]:
import os
import re
import fitz
import torch
import random
import requests
import textwrap
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from spacy.lang.en import English
from transformers.utils import is_flash_attn_2_available
from sentence_transformers import util, SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM